In [1]:
import pyomo.environ as pyo
import numpy as np
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

In [2]:
elements = ['a', 'b', 'c', 'e', 'g',
               'h', 'j', 'k', 'm', 'n',
               'o', 'p', 'q', 'r', 's']



In [3]:
class LogDetModel(ExternalGreyBoxModel):
    def __init__(self, initial_values, use_exact_derivatives=True,verbose=False):
        self._use_exact_derivatives = use_exact_derivatives
        self.verbose = verbose
        self.initial = initial_values

        # For use with exact Hessian
        self._output_con_mult_values = np.zeros(1)

        if not use_exact_derivatives:
            raise NotImplementedError("use_exact_derivatives == False not supported")
        
    def input_names(self):
        return ['a', 'b', 'c', 'e', 'g',
               'h', 'j', 'k', 'm', 'n',
               'o', 'p', 'q', 'r', 's']

    def equality_constraint_names(self):
        # no equality constraints
        return [ ]
    
    def output_names(self):
        return ['log_det']

    def set_output_constraint_multipliers(self, output_con_multiplier_values):
        assert len(output_con_multiplier_values) == 1
        np.copyto(self._output_con_mult_values, output_con_multiplier_values)

    def finalize_block_construction(self, pyomo_block):
        # initialize, set up LB and UB
        for i, ele in enumerate(elements):
            pyomo_block.inputs[ele].setlb(self.initial[i])
            pyomo_block.inputs[ele].setub(self.initial[i])
        
        

    def set_input_values(self, input_values):
        self._input_values = list(input_values)

    def evaluate_equality_constraints(self):

        # Not sure what this function should return with no equality constraints
        return None
    
    def evaluate_outputs(self):
        a = self._input_values[0]
        b = self._input_values[1]
        c = self._input_values[2]
        e = self._input_values[3]
        g = self._input_values[4]
        
        h = self._input_values[5]
        j = self._input_values[6]
        k = self._input_values[7]
        m = self._input_values[8]
        n = self._input_values[9] 
        
        o = self._input_values[10]
        p = self._input_values[11]
        q = self._input_values[12]
        r = self._input_values[13]
        s = self._input_values[14]

        # form matrix
        M = np.array([[a,h,j,k,m],
                      [h,b,n,o,p],
                     [j,n,c,q,r],
                     [k,o,q,e,s],
                     [m,p,r,s,g]])

        # compute log determinant
        (sign, logdet) = np.linalg.slogdet(M)

        if self.verbose:
            print("\n Consider M =\n",M)
            print("   logdet = ",logdet,"\n")

        return np.asarray([logdet], dtype=np.float64)

    def evaluate_jacobian_equality_constraints(self):
        return None

    def evaluate_jacobian_outputs(self):

        a = self._input_values[0]
        b = self._input_values[1]
        c = self._input_values[2]
        e = self._input_values[3]
        g = self._input_values[4]
        
        h = self._input_values[5]
        j = self._input_values[6]
        k = self._input_values[7]
        m = self._input_values[8]
        n = self._input_values[9] 
        
        o = self._input_values[10]
        p = self._input_values[11]
        q = self._input_values[12]
        r = self._input_values[13]
        s = self._input_values[14]

        if self._use_exact_derivatives:

            M = np.array([[a,h,j,k,m],
                      [h,b,n,o,p],
                     [j,n,c,q,r],
                     [k,o,q,e,s],
                     [m,p,r,s,g]])

            # compute inverse
            Minv = np.linalg.pinv(M)

            row = np.zeros(15)
            col = np.zeros(15)
            data = np.zeros(15)
            row[0], col[0], data[0] = (0, 0, Minv[0,0])
            row[1], col[1], data[1] = (0, 1, Minv[1,1])
            row[2], col[2], data[2] = (0, 2, Minv[2,2])
            row[3], col[3], data[3] = (0, 3, Minv[3,3])
            row[4], col[4], data[4] = (0, 4, Minv[4,4])
            row[5], col[5], data[5] = (0, 5, 2*Minv[0,1])
            row[6], col[6], data[6] = (0, 6, 2*Minv[0,2])
            row[7], col[7], data[7] = (0, 7, 2*Minv[0,3])
            row[8], col[8], data[8] = (0, 8, 2*Minv[0,4])
            row[9], col[9], data[9] = (0, 9, 2*Minv[1,2])
            row[10], col[10], data[10] = (0, 10, 2*Minv[1,3])
            row[11], col[11], data[11] = (0, 11, 2*Minv[1,4])
            row[12], col[12], data[12] = (0, 12, 2*Minv[2,3])
            row[13], col[13], data[13] = (0, 13, 2*Minv[2,4])
            row[14], col[14], data[14] = (0, 14, 2*Minv[3,4])
            return coo_matrix((data, (row, col)), shape=(1,15))


In [4]:
def maximize_log_det(show_solver_log=True, additional_options={}):
    m = pyo.ConcreteModel()
    
    #initial = [10530, 146, 342434, 0.53, 34352,
    #      134525,3232,62453,2923,635423,
    #      83343,93432,14353,3893255,4436354]
    initial = [22.290702329071145, 0.050382192666572484, 11.342097279294373, 
               125.93205407126952, 12.149207865794569, 0.2107437823246938, 
               -8.805387284249873, 35.32791654214199, -10.271277335786364, 
               0.0487656762611641, 1.140357302639742, -0.17206501821435413, 
               -24.843745501736198, -3.0682747332989853, -3.625239981761326]
    
    # create a block to store the external grey box model
    m.my_block = ExternalGreyBoxBlock(external_model=LogDetModel(initial_values = initial))
    
    # add objective to maximize log det
    m.obj = pyo.Objective(expr=m.my_block.outputs['log_det'], sense=pyo.maximize)

    solver = pyo.SolverFactory('cyipopt')

    solver.config.options['hessian_approximation'] = 'limited-memory'

    for k,v in additional_options.items():
        solver.config.options[k] = v
    
    solver.tee=True
        
    
    results = solver.solve(m, tee=True)
    pyo.assert_optimal_termination(results)
    return m, solver

In [5]:
m, solver1 = maximize_log_det(additional_options={'max_iter':500})


In [6]:
print("Grey-box log det:", m.my_block.outputs['log_det'].value)

a = m.my_block.inputs['a'].value
b = m.my_block.inputs['b'].value
c = m.my_block.inputs['c'].value
e = m.my_block.inputs['e'].value
g = m.my_block.inputs['g'].value
h = m.my_block.inputs['h'].value
j = m.my_block.inputs['j'].value
k = m.my_block.inputs['k'].value
mm = m.my_block.inputs['m'].value
n = m.my_block.inputs['n'].value
o = m.my_block.inputs['o'].value
p = m.my_block.inputs['p'].value
q = m.my_block.inputs['q'].value
r = m.my_block.inputs['r'].value
s = m.my_block.inputs['s'].value

test = np.asarray([[a,h,j,k,mm],
                   [h,b,n,o,p],
                   [j,n,c,q,r],
                  [k,o,q,e,s],
                  [mm,p,r,s,g]])
print(test)

det = np.linalg.det(test)
print('Real log det:', np.log(det))

Grey-box log det: 5.405453593297799
[[ 2.22907023e+01  2.10743782e-01 -8.80538728e+00  3.53279165e+01
  -1.02712773e+01]
 [ 2.10743782e-01  5.03821927e-02  4.87656763e-02  1.14035730e+00
  -1.72065018e-01]
 [-8.80538728e+00  4.87656763e-02  1.13420973e+01 -2.48437455e+01
  -3.06827473e+00]
 [ 3.53279165e+01  1.14035730e+00 -2.48437455e+01  1.25932054e+02
  -3.62523998e+00]
 [-1.02712773e+01 -1.72065018e-01 -3.06827473e+00 -3.62523998e+00
   1.21492079e+01]]
Real log det: 5.405453593297799
